## Better Way 41. 진정한 병렬성을 실행하려면 concurrent.futures를 고려하자

* **파이썬 프로그램에서 성능 충족하기**
	- 코드 최적화 이후에도 실행 속도가 매우 느릴 수 있음.
    - **병렬성**이 한 해결책이지만...
    	+ ~~코드의 연산 부분을 여러 CPU 코어에서 동시에 실행할 수 있게 독립적으로 동작하는 부분으로 나누기~~
            + **전역 인터프리터 잠금(GIL)이 스레드에서 진정한 병렬성을 막기 때문에(**BW 37**) 불가능.
        + **가장 성능이 중요한 코드를 C 언어 확장 모듈로 재작성하기**
        	+ 일반적 방법.
            + 하드웨어에 더 가까워지고 파이썬보다 빨리 실행할 수 있어 병렬화할 필요가 사라짐.
            + 하지만 코드 재작성 비용 및 버그 문제도 고려해야 함.
            + C 확장으로 병렬로 실행하는 네이티브 스레드를 시작해서 여러 CPU 코어를 활용할 수 있음.
            + C로의 변환 작업을 수월하게 해주는 오픈 소스 도구 존재.
                + Cython(http://cython.org)
                + Numba(http://numba.pydata.org/)
        + **내장 모듈 multiprocessing 사용**
            + 파이썬에서 특정 유형의 계산을 최소한의 노력으로 병렬화할 수 있는 방법.
            + **`concurrent.futures`**로 쉽게 접근 가능.
            + 자식 프로세스로 추가 인터프리터를 실행해 병렬로 여러 CPU 코어 활용 가능.
                + **자식 프로세스는 주 인터프리터와 별개이므로 GIL 역시 분리됨.**
                + 각 자식은 CPU 코어 하나를 온전히 사용 가능.

### 예시: 두 숫자의 최대공약수를 찾는 알고리즘 구현

#### I. 병렬성이 없는 경우.

In [1]:
def gcd(pair):
    a, b = pair
    low = min(a, b)
    for i in range(low, 0, -1):
        if a % i == 0 and b % i == 0:
            return i

병렬성이 없으므로 gcd 함수를 순서대로 실행하면 시간이 선형적으로 증가함.

In [2]:
from time import time
numbers = [(1963309, 2265973), (2030677, 3814172),
           (1551645, 2229620), (2039045, 2020802)]
start = time()
results = list(map(gcd, numbers))
end = time()
print('Took %.3f seconds' % (end - start))

Took 0.479 seconds


여러 파이썬 스레드에서 위 코드를 실행하면 GIL로 인해 여러 CPU 코어의 병렬 사용이 불가해 속도가 개선 안 됨.

#### II. `concurrents.futures` 모듈의 `ThreadPoolExecutor` 클래스 및 작업 스레드 2개를 사용하는 경우.

In [3]:
from concurrent.futures import ThreadPoolExecutor

start = time()
pool = ThreadPoolExecutor(max_workers=2)
results = list(pool.map(gcd, numbers))
end = time()
print('Took %.3f seconds' % (end - start))

Took 0.485 seconds


위 결과는 **스레드 풀 시작 및 통신에 드는 오버헤드**로 인해 더 느림.

#### III. `ThreadPoolExecutor`를 `concurrent.futures` 모듈의 `ProcessPoolExecutor`로 대체하는 경우.

In [4]:
from multiprocessing import Process, freeze_support

def foo():
    print('hello')

if __name__ == '__main__':
    freeze_support()
    p = Process(target=foo)
    p.start()

In [5]:
from concurrent.futures import ProcessPoolExecutor

start = time()
pool = ProcessPoolExecutor(max_workers=2)  # The one change
results = list(pool.map(gcd, numbers))
end = time()
print('Took %.3f seconds' % (end - start))

BrokenProcessPool: A process in the process pool was terminated abruptly while the future was running or pending.

책의 코드는 몇 가지 문제로 Windows Jupyter Notebook 환경에서 그대로 실행되지 않음.
* 파이썬 3.7의 `multiprocessing` 문서 참조.
    + `Pool` 클래스(**작업자 프로세스 풀**을 나타냄)를 사용하기 위해서는 `__main__` 모듈을 자식 프로세스가 임포트할 수 있어야 함.
        + 다른 모듈과 구분되는 `__main__` 모듈이 존재해야 함.
        + 대화형 인터프리터에서는 동작하지 않음.
        + 새 프로세스 시작 등의 부작용을 일으키지 않고 `__main__` 모듈을 안전하게 가져오기 위해서는 `if __name__ == '__main__'`을 사용해 프로그램의 **진입 지점entry point**을 보호해야 함.
* `if __name__ == '__main__'`을 사용한 .py 파일을 %run으로 실행하더라도 Jupyter Notebook에서는 다음과 같은 오류가 발생함: `AttributeError: module '__main__' has no attribute '__spec__'`.
    + `if __name__ == '__main__'` 하단에 `__spec__ = "ModuleSpec(name='builtins', loader=<class '_frozen_importlib.BuiltinImporter'>)"`를 추가하면 정상적으로 실행됨.


* 관련 내용은 아래 링크들 참조.
    + https://docs.python.org/3/library/multiprocessing.html#multiprocessing-programming
    + https://stackoverflow.com/questions/14175348/why-does-pythons-multiprocessing-module-import-main-when-starting-a-new-pro
    + https://stackoverflow.com/questions/45720153/python-multiprocessing-error-attributeerror-module-main-has-no-attribute

In [6]:
%run BetterWay_41_YChoi_ProcessPoolExecutor

Took 0.410 seconds


In [7]:
# from concurrent.futures import ProcessPoolExecutor
# from time import time

# def gcd(pair):
#     a, b = pair
#     low = min(a, b)
#     for i in range(low, 0, -1):
#         if a % i == 0 and b % i == 0:
#             return i

# def main():
#     numbers = [(1963309, 2265973), (2030677, 3814172),
#                (1551645, 2229620), (2039045, 2020802)]
#     start = time()
#     pool = ProcessPoolExecutor(max_workers=2)  # The one change
#     results = list(pool.map(gcd, numbers))
#     end = time()
#     print('Took %.3f seconds' % (end - start))

# if __name__ == "__main__":
#     __spec__ = "ModuleSpec(name='builtins', loader=<class '_frozen_importlib.BuiltinImporter'>)"
#     main()

속도가 향상되는 것을 확인 가능.

### ProcessPoolExecutor 클래스가 (multiprocessing 모듈이 제공하는 저수준 구조를 이용해) 실제로 하는 작업

1. numbers 입력 데이터에서 map으로 각 아이템을 가져온다.
2. pickle 모듈을 사용하여 바이너리 데이터로 직렬화한다(**BW44**).
3. 주 인터프리터 프로세스에서 직렬화한 데이터를 지역 소켓을 통해 자식 인터프리터 프로세스로 복사한다.
4. 자식 프로세스에서 pickle을 사용하여 데이터를 파이썬 객체로 역직렬화한다.
5. gcd 함수가 들어 있는 파이썬 모듈을 임포트한다.
6. 다른 자식 프로세스를 사용하여 병렬로 입력 데이터에 함수를 실행한다.
7. 결과를 다시 바이트로 직렬화한다.
8. 소켓을 통해 바이트를 다시 복사한다.
9. 바이트를 부모 프로세스에 있는 파이썬 객체로 역직렬화한다.
10. 마지막으로 여러 자식에 있는 결과를 반환용 리스트 한 개로 합친다.


* `multiprocessing`
    - 부모와 자식 프로세스 간에 일어날 수밖에 없는 모든 직렬화와 역직렬화 때문에 비용이 높음.
    - 고립되고 지렛대 효과가 큰 특정 유형의 작업에 적합.
        + **고립(isolated)**: 프로그램의 다른 부분과 상태를 공유할 필요가 없음.
        + **지렛대 효과가 큼(high-leverage)**: 부모와 자식 프로세스 사이에서 데이터를 조금만 전송해도 많은 양의 계산이 일어나야 함.
            + 최대공약수 계산 알고리즘이 한 예.
            + 이외의 다른 수학적 알고리즘도 대부분 비슷하게 동작.
    - 이러한 유형의 계산이 아닌 경우?
        + `multiprocessing` 비용이 병렬성을 통한 속도 향상을 막을 수 있음.
        + `mutiprocessing`은 이런 상황에 쓸 수 있는 공유 메모리, 프로세스 간 잠금, 큐, 프록시 같은 고급 기능을 제공하지만 지나치게 복잡함.
    - 결론?
        + **`multiprocessing`의 모든 기능을 직접 사용하지 말고, 더 간단한 `concurrent.futures` 모듈을 통해 사용하는 것을 권장.**
            + `ThreadPoolExecutor` 클래스를 사용하여 고립되고 지렛대 효과가 큰 함수를 스레드에서 실행하는 것부터 시작.
            + 추후 속도를 향상하려면 `ProcessPoolExecutor`클래스를 사용.
            + `multiprocessing` 모듈의 직접 사용은 마지막으로 고려.
            

## 핵심 정리
* CPU 병목점을 C 확장 모듈로 옮기기:
    - 장점: 파이썬 코드에 최대한 투자하면서 성능을 개선할 수 있는 효과적인 방법.
    - 단점: 비용이 많이 들어가고 버그가 생길 수 있음.
* 다른 해결책: `multiprocessing` 모듈
    - 파이썬에서 특정 유형의 계산을 최소한의 노력으로 병렬화할 수 있는 강력한 도구를 제공.
* `multiprocessing`의 강력한 기능은 `concurrent.futures` 및 그 안의 간단한 `ProcessPoolExecutor` 클래스로 접근하는 게 가장 좋음.
* `multiprocessing` 모듈의 고급 기능은 너무 복잡하므로 피하는 것을 권장.